In [1]:
import os
os.environ['TF_DEVICE_MIN_SYS_MEMORY_IN_MB'] = '128' 

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
from huggingface_hub import notebook_login

notebook_login(new_session=False)

User is already logged in.


In [3]:
language = 'bn'
model_str = 'whisper-tiny'
dataset_folder = "/media/ahanaf/media-1/Datasets/ben10/ben10/"


In [4]:
from datasets import load_dataset, IterableDatasetDict, DatasetDict, Features, Value

ben10 = IterableDatasetDict()


# Load the ASR dataset using load_dataset
ben10['train'] = load_dataset(
    'csv', 
    data_files={
        'train': f'{dataset_folder}/cleaned_train.csv'
    },
    split='train', 
    streaming=True,
)

ben10['valid'] = load_dataset(
    'csv', 
    data_files={
        'valid': f'{dataset_folder}/cleaned_val.csv'
    },
    split='valid', 
    streaming=True,
)

print(ben10)



IterableDatasetDict({
    train: IterableDataset({
        features: Unknown,
        n_shards: 1
    })
    valid: IterableDataset({
        features: Unknown,
        n_shards: 1
    })
})


In [5]:
import librosa
import numpy as np

def load_audio(audio_path, sr=16000):
    # Load audio file
    audio_data, sr = librosa.load(audio_path, sr=sr)
    return audio_data, sr

def add_path(batch):
    batch['path'] = f'{dataset_folder}/16_kHz_train_audio/' + batch['file_name']
    batch['sentence'] = batch['transcripts_clean']
    batch['audio'] = {}
    out = load_audio(batch["path"], sr=16000)
    batch['audio']['array'] = out[0].astype(np.float64)
    batch['audio']['sampling_rate'] = out[1]
    return batch

ben10 = ben10.map(add_path)
ben10 = ben10.remove_columns(["len", "transcripts", 'file_name', 'transcripts_clean'])

for x in ben10['train']:
    break
x

{'path': '/media/ahanaf/media-1/Datasets/ben10/ben10//16_kHz_train_audio/train_habiganj (113).wav',
 'sentence': 'ওইবো সমস্যা নাই। আমরার খতা ওইছে খালি গ্রামের ভাষা দরখার, তুমি কিতা খইতাছো, না খইতাছো ইডা ব্যাফার না৷ বুচ্ছো? খও, শুরু খরো। তে অখন খও তোমার জামাইর বাড়ি কিরখম লাগে?',
 'audio': {'array': array([-2.44140625e-04, -9.46044922e-04, -9.76562500e-04, ...,
         -9.15527344e-05, -6.10351562e-05,  0.00000000e+00]),
  'sampling_rate': 16000}}

In [6]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor


feature_extractor = WhisperFeatureExtractor.from_pretrained(f"openai/{model_str}")
tokenizer = WhisperTokenizer.from_pretrained(f"openai/{model_str}", language='Bengali', task="transcribe")
processor = WhisperProcessor.from_pretrained(f"openai/{model_str}", language="Bengali", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
for x in ben10['train']:
    arr = x['audio']['array']
    data_sample = x
    print(arr.min(), arr.max())
    break
    

-0.873291015625 0.991729736328125


In [8]:
input_str = data_sample["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 ওইবো সমস্যা নাই। আমরার খতা ওইছে খালি গ্রামের ভাষা দরখার, তুমি কিতা খইতাছো, না খইতাছো ইডা ব্যাফার না৷ বুচ্ছো? খও, শুরু খরো। তে অখন খও তোমার জামাইর বাড়ি কিরখম লাগে?
Decoded w/ special:    <|startoftranscript|><|bn|><|transcribe|><|notimestamps|>ওইবো সমস্যা নাই। আমরার খতা ওইছে খালি গ্রামের ভাষা দরখার, তুমি কিতা খইতাছো, না খইতাছো ইডা ব্যাফার না৷ বুচ্ছো? খও, শুরু খরো। তে অখন খও তোমার জামাইর বাড়ি কিরখম লাগে?<|endoftext|>
Decoded w/out special: ওইবো সমস্যা নাই। আমরার খতা ওইছে খালি গ্রামের ভাষা দরখার, তুমি কিতা খইতাছো, না খইতাছো ইডা ব্যাফার না৷ বুচ্ছো? খও, শুরু খরো। তে অখন খও তোমার জামাইর বাড়ি কিরখম লাগে?
Are equal:             True


In [9]:
print(ben10["train"])

IterableDataset({
    features: Unknown,
    n_shards: 1
})


In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    # print(batch)
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"], max_length=400).input_ids
    return batch

In [11]:
mapped_ben10 = ben10.map(prepare_dataset)

for xx in mapped_ben10['train']:
    break

print(xx['input_features'].shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(80, 3000)


In [12]:
mapped_ben10['train']

IterableDataset({
    features: Unknown,
    n_shards: 1
})

In [13]:
# for x in mapped_common_voice['train']:
#     print(x)
#     break

In [14]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
import evaluate

metric = evaluate.load("wer")

In [17]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [18]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(f"openai/{model_str}")

In [19]:
model.generation_config.language = "bengali"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [21]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_str}-{language}-ben10",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=15000,
    eval_delay=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=410,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    # push_to_hub=True,
    # dataloader_prefetch_factor=512,
    # dataloader_num_workers=128,
    auto_find_batch_size=True,
    # hub_private_repo=
    resume_from_checkpoint=False,
)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=mapped_ben10["train"],
    eval_dataset=mapped_ben10["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer.train(resume_from_checkpoint=False)

  0%|          | 0/15000 [00:00<?, ?it/s]

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


{'loss': 2.5618, 'grad_norm': 55.716739654541016, 'learning_rate': 4.2000000000000006e-07, 'epoch': 0.0}
{'loss': 2.2894, 'grad_norm': 21.152835845947266, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.0}
{'loss': 2.0056, 'grad_norm': 10.836516380310059, 'learning_rate': 1.42e-06, 'epoch': 0.01}
{'loss': 1.8114, 'grad_norm': 7.599587440490723, 'learning_rate': 1.9200000000000003e-06, 'epoch': 0.01}
{'loss': 1.6671, 'grad_norm': 6.7855401039123535, 'learning_rate': 2.42e-06, 'epoch': 0.01}
{'loss': 1.5701, 'grad_norm': 8.88961124420166, 'learning_rate': 2.92e-06, 'epoch': 0.01}
{'loss': 1.5002, 'grad_norm': 7.530593395233154, 'learning_rate': 3.4200000000000007e-06, 'epoch': 0.01}
{'loss': 1.4482, 'grad_norm': 5.5929059982299805, 'learning_rate': 3.920000000000001e-06, 'epoch': 0.01}
{'loss': 1.4111, 'grad_norm': 8.489799499511719, 'learning_rate': 4.42e-06, 'epoch': 0.01}
{'loss': 1.3826, 'grad_norm': 9.671767234802246, 'learning_rate': 4.92e-06, 'epoch': 0.02}
{'loss': 1.3621, 'gr

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.1443, 'grad_norm': 9.077363967895508, 'learning_rate': 9.916551724137932e-06, 'epoch': 0.04}
{'loss': 1.1379, 'grad_norm': 12.91983699798584, 'learning_rate': 9.899310344827588e-06, 'epoch': 0.04}
{'loss': 1.1135, 'grad_norm': 6.9064741134643555, 'learning_rate': 9.882068965517241e-06, 'epoch': 0.04}
{'loss': 1.1222, 'grad_norm': 11.685742378234863, 'learning_rate': 9.864827586206898e-06, 'epoch': 0.05}
{'loss': 1.1051, 'grad_norm': 8.048666954040527, 'learning_rate': 9.847586206896553e-06, 'epoch': 0.05}


...

{'loss': 0.3734, 'grad_norm': 7.457297325134277, 'learning_rate': 4.246206896551725e-06, 'epoch': 12.0}
{'loss': 0.3588, 'grad_norm': 5.426616668701172, 'learning_rate': 4.22896551724138e-06, 'epoch': 12.0}
{'loss': 0.3706, 'grad_norm': 6.948919773101807, 'learning_rate': 4.2117241379310345e-06, 'epoch': 12.0}
{'loss': 0.3576, 'grad_norm': 7.527602195739746, 'learning_rate': 4.19448275862069e-06, 'epoch': 12.01}
{'loss': 0.3781, 'grad_norm': 4.859462261199951, 'learning_rate': 4.177241379310345e-06, 'epoch': 12.01}
{'loss': 0.3735, 'grad_norm': 6.620262622833252, 'learning_rate': 4.16e-06, 'epoch': 12.01}
{'loss': 0.3518, 'grad_norm': 5.711750030517578, 'learning_rate': 4.142758620689656e-06, 'epoch': 12.01}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.49958541989326477, 'eval_wer': 110.80174680917798, 'eval_runtime': 460.6077, 'eval_samples_per_second': 2.842, 'eval_steps_per_second': 0.178, 'epoch': 12.01}


/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3575, 'grad_norm': 4.740829944610596, 'learning_rate': 4.1255172413793106e-06, 'epoch': 12.01}
{'loss': 0.3649, 'grad_norm': 7.27004337310791, 'learning_rate': 4.108275862068966e-06, 'epoch': 12.01}
{'loss': 0.3557, 'grad_norm': 5.647800445556641, 'learning_rate': 4.091034482758621e-06, 'epoch': 12.02}
{'loss': 0.354, 'grad_norm': 5.741909027099609, 'learning_rate': 4.073793103448276e-06, 'epoch': 12.02}
{'loss': 0.3533, 'grad_norm': 5.247207164764404, 'learning_rate': 4.056551724137932e-06, 'epoch': 12.02}
{'loss': 0.3665, 'grad_norm': 5.117321014404297, 'learning_rate': 4.039310344827587e-06, 'epoch': 12.02}
{'loss': 0.3536, 'grad_norm': 5.176582336425781, 'learning_rate': 4.0220689655172415e-06, 'epoch': 12.02}
{'loss': 0.3525, 'grad_norm': 4.976435661315918, 'learning_rate': 4.004827586206897e-06, 'epoch': 12.02}
{'loss': 0.3418, 'grad_norm': 6.420814037322998, 'learning_rate': 3.987586206896552e-06, 'epoch': 12.03}
{'loss': 0.3598, 'grad_norm': 5.498348712921143, 'learn

In [ ]:
import torch

In [ ]:
torch.cuda.memory_stats()

OrderedDict([('active.all.allocated', 284789737),
             ('active.all.current', 503),
             ('active.all.freed', 284789234),
             ('active.all.peak', 775),
             ('active.large_pool.allocated', 31463846),
             ('active.large_pool.current', 57),
             ('active.large_pool.freed', 31463789),
             ('active.large_pool.peak', 106),
             ('active.small_pool.allocated', 253325891),
             ('active.small_pool.current', 446),
             ('active.small_pool.freed', 253325445),
             ('active.small_pool.peak', 698),
             ('active_bytes.all.allocated', 302894913586176),
             ('active_bytes.all.current', 465560064),
             ('active_bytes.all.freed', 302894448026112),
             ('active_bytes.all.peak', 4381111808),
             ('active_bytes.large_pool.allocated', 256030036684288),
             ('active_bytes.large_pool.current', 376891904),
             ('active_bytes.large_pool.freed', 2560296597923